# Vector Similarity Search Extension

`vss`扩展是DuckDB的一个实验性扩展，它通过DuckDB的新固定大小`ARRAY`类型，为向量相似性搜索查询添加索引支持以加速查询。

#### Usage

- **创建HNSW索引**：使用`CREATE INDEX`语句和`USING HNSW`子句，在具有`ARRAY`列的表上创建新的HNSW（Hierarchical Navigable Small Worlds）索引。例如：

In [12]:
# 确保 conn 是你在 notebook 环境中已创建的 DuckDB 连接对象

conn.execute("INSTALL vss;")  # 安装 vss 扩展
conn.execute("LOAD vss;")     # 加载 vss 扩展

conn.execute("DROP TABLE IF EXISTS my_vector_table;")  # 如果表已存在则删除
conn.execute("CREATE TABLE my_vector_table (vec FLOAT[3]);")  # 创建包含向量列的表
conn.execute("""
INSERT INTO my_vector_table
    SELECT array_value(a, b, c)
    FROM range(1, 10) ra(a), range(1, 10) rb(b), range(1, 10) rc(c);
""")  # 插入三维向量数据
conn.execute("CREATE INDEX my_hnsw_index ON my_vector_table USING HNSW (vec);")  # 在 vec 列上创建 HNSW 索引
print("已创建表 'my_vector_table' 并构建 HNSW 索引 'my_hnsw_index'.")


已创建表 'my_vector_table' 并构建 HNSW 索引 'my_hnsw_index'.


- **加速查询**：该索引可用于加速使用`ORDER BY`子句评估支持的距离度量函数与索引列和常量向量之间的距离，并以`LIMIT`子句结尾的查询。例如：

In [13]:
# 查询与 [1, 2, 3] 最近的 3 个向量
result = conn.execute("""
SELECT *
FROM my_vector_table
ORDER BY array_distance(vec, [1, 2, 3]::FLOAT[3])
LIMIT 3;
""").fetchall()
print(result)  # 打印查询结果

[((1.0, 2.0, 3.0),), ((2.0, 2.0, 3.0),), ((1.0, 2.0, 4.0),)]


- **使用`min_by`函数**：如果`arg`参数是匹配的距离度量函数，`min_by(col, arg, n)`的重载版本也可以通过`HNSW`索引加速，可用于快速一次性最近邻搜索。例如，获取与`[1, 2, 3]`最接近的前3行：

In [17]:
# 使用 min_by 函数查找与 [1, 2, 3] 最近的 3 个向量
min_by_result = conn.execute("""
SELECT
    -- min_by(table, 距离度量, 返回前n个)
    min_by(
        my_vector_table,  -- 要返回的表行
        array_distance(vec, [1, 2, 3]::FLOAT[3]),  -- 计算 vec 与 [1,2,3] 的欧氏距离
        3  -- 返回距离最近的3个
    ) AS result
FROM
    my_vector_table;  -- 查询的数据表
""").fetchall()

if min_by_result and min_by_result[0]:
    print(min_by_result[0][0])
else:
    print("No result from min_by query.")

[{'vec': (1.0, 2.0, 3.0)}, {'vec': (2.0, 2.0, 3.0)}, {'vec': (1.0, 2.0, 4.0)}]


- **验证索引使用**：通过检查`EXPLAIN`输出并查找计划中的`HNSW_INDEX_SCAN`节点，可以验证索引是否被使用。例如：

In [43]:
explain_output = conn.execute("""
EXPLAIN
SELECT *
FROM my_vector_table  -- 查询向量表
ORDER BY array_distance(vec, [1, 2, 3]::FLOAT[3])  -- 按与 [1,2,3] 的距离升序排序
LIMIT 3;  -- 只取最近的3个
""").fetchall()
index = 1
for row in explain_output:
    print(row)
    print(f"Row {index}: {row[0]}")
    index += 1
    
    

('physical_plan', '┌───────────────────────────┐\n│         PROJECTION        │\n│    ────────────────────   │\n│             #0            │\n│                           │\n│          ~3 Rows          │\n└─────────────┬─────────────┘\n┌─────────────┴─────────────┐\n│         PROJECTION        │\n│    ────────────────────   │\n│            vec            │\n│            NULL           │\n│                           │\n│         ~729 Rows         │\n└─────────────┬─────────────┘\n┌─────────────┴─────────────┐\n│      HNSW_INDEX_SCAN      │\n│    ────────────────────   │\n│           Table:          │\n│      my_vector_table      │\n│                           │\n│        HSNW Index:        │\n│       my_hnsw_index       │\n│                           │\n│      Projections: vec     │\n│                           │\n│         ~729 Rows         │\n└───────────────────────────┘\n')
Row 1: physical_plan


- **指定距离度量**：默认情况下，HNSW索引使用欧几里得距离`l2sq`（L2范数平方）度量，与DuckDB的`array_distance`函数匹配，但也可以在创建索引时指定其他距离度量。例如：

In [25]:
# 删除已存在的索引（如果有）
conn.execute("""
DROP INDEX IF EXISTS my_hnsw_cosine_index;
""")

# 创建使用余弦相似性度量的 HNSW 索引
conn.execute("""
CREATE INDEX my_hnsw_cosine_index  -- 创建名为 my_hnsw_cosine_index 的索引
ON my_vector_table                 -- 在 my_vector_table 表上
USING HNSW (vec)                   -- 使用 HNSW 索引方法，索引 vec 列
WITH (
    metric = 'cosine'              -- 指定使用余弦相似性作为距离度量
);
""")

print("已创建索引 'my_hnsw_cosine_index'，在 'my_vector_table' 上使用余弦相似度度量。")

已创建索引 'my_hnsw_cosine_index'，在 'my_vector_table' 上使用余弦相似度度量。


支持的距离度量及其对应的DuckDB函数如下表所示：
| Metric| Function| Description|
| ---| ---| ---|
| `l2sq`| `array_distance`| 欧几里得距离|
| `cosine`| `array_cosine_distance`| 余弦相似性距离|
| `ip`| `array_negative_inner_product`| 负内积|

- **多索引支持**：虽然每个`HNSW`索引仅适用于单个列，但可以在同一表上创建多个`HNSW`索引，每个索引分别索引不同的列。此外，也可以为同一列创建多个`HNSW`索引，每个索引支持不同的距离度量。

#### Index Options

除了`metric`选项外，`HNSW`索引创建语句还支持以下选项，用于控制索引构建和搜索过程的超参数：

| Option| Default| Description|
| ---| ---| ---|
| `ef_construction`| 128| 在构建索引时考虑的候选顶点数量。更高的值将使索引更准确，但也会增加构建索引所需的时间。|
| `ef_search`| 64| 在索引的搜索阶段考虑的候选顶点数量。更高的值将使索引更准确，但也会增加搜索所需的时间。|
| `M`| 16| 图中每个顶点保持的最大邻居数量。更高的值将使索引更准确，但也会增加构建索引所需的时间。|
| `M0`| 2 * `M`| 零级图中每个顶点保持的邻居数量。更高的值将使索引更准确，但也会增加构建索引所需的时间。|

此外，还可以在运行时通过设置`SET hnsw_ef_search = int`配置选项来覆盖在索引构建时设置的`ef_search`参数。如果希望在每个连接的基础上权衡搜索性能和准确性，这将非常有用。也可以通过调用`RESET hnsw_ef_search`来取消覆盖。

In [32]:
# 设置 hnsw_ef_search 参数以控制搜索精度和性能的平衡
conn.execute("""
SET hnsw_ef_search = 100;  -- 将搜索阶段考虑的候选顶点数量增加到100，提高搜索准确性但可能降低查询速度
""")

# 查询当前 hnsw_ef_search 参数值
res = conn.execute("""
SELECT 
    current_setting('hnsw_ef_search')  -- 获取当前 hnsw_ef_search 配置参数的值
""").fetchone()[0]
print(f"hnsw_ef_search set to: {res}")

# 将 hnsw_ef_search 参数恢复到默认值
conn.execute("""
RESET hnsw_ef_search;  -- 重置参数到系统默认值（通常为64），恢复默认的搜索性能与准确性平衡
""")

# 验证参数是否已成功重置
res2 = conn.execute("""
SELECT 
    current_setting('hnsw_ef_search')  -- 检查重置后的参数当前值
""").fetchone()[0]
print(f"hnsw_ef_search reset. Current value: {res2}")

# 注意：默认值可能取决于DuckDB版本，如果未设置，可能会返回错误或默认值。
# 根据VSS文档说明，ef_search在索引创建时的默认值为64。

hnsw_ef_search set to: 100
hnsw_ef_search reset. Current value: None


#### Persistence

由于一些已知的与自定义扩展索引持久化相关的问题，默认情况下，`HNSW`索引只能在内存数据库的表上创建，除非将`SET hnsw_enable_experimental_persistence = bool`配置选项设置为`true`。

将此功能锁定在实验性标志后面的原因是“WAL”恢复尚未为自定义索引正确实现，这意味着如果在`HNSW`索引表上有未提交的更改时发生崩溃或数据库意外关闭，可能会导致**数据丢失或索引损坏**。

如果启用此选项并遇到意外关闭，可以通过首先单独启动DuckDB，加载`vss`扩展，然后`ATTACH`数据库文件来尝试恢复索引，这确保了在WAL回放期间`HNSW`索引功能可用，从而使DuckDB的恢复过程能够顺利进行。但仍然建议不要在生产环境中使用此功能。

启用`hnsw_enable_experimental_persistence`选项后，索引将被持久化到DuckDB数据库文件中（如果以磁盘支持的数据库文件运行DuckDB），这意味着在数据库重新启动后，可以从磁盘将索引加载回内存，而无需重新创建。需要注意的是，没有对持久化索引存储的增量更新，因此每次DuckDB执行检查点时，整个索引将被序列化到磁盘并覆盖自身。同样，在数据库重新启动后，索引将被完整地反序列化回主内存。尽管这将被延迟到首次访问与索引相关的表时，但根据索引的大小，反序列化过程可能需要一些时间，不过它仍然应该比简单地删除并重新创建索引更快。

In [34]:
# 启用 HNSW 索引的实验性持久化功能
conn.execute("""
SET hnsw_enable_experimental_persistence = true;  -- 允许将 HNSW 索引持久化到磁盘
""")

# 查询当前持久化设置并显示
persistence_status = conn.execute("""
SELECT 
    current_setting('hnsw_enable_experimental_persistence')  -- 获取当前持久化设置的值
""").fetchone()[0]
print(f"HNSW 索引实验性持久化功能已启用: {persistence_status}")

# 将 HNSW 索引持久化设置重置为默认值（禁用）
# 注意：在生产环境中，由于WAL恢复机制的限制，建议保持禁用状态以避免数据丢失风险
conn.execute("""
SET hnsw_enable_experimental_persistence = false;  -- 禁用 HNSW 索引持久化功能
""")

# 再次查询设置以确认已重置
persistence_status_reset = conn.execute("""
SELECT 
    current_setting('hnsw_enable_experimental_persistence')  -- 检查设置是否成功重置
""").fetchone()[0]
print(f"HNSW 索引实验性持久化功能已重置为: {persistence_status_reset}")

# 注意：启用持久化功能可能导致以下风险：
# 1. 在有未提交更改时数据库意外关闭可能导致数据丢失或索引损坏
# 2. 每次检查点时，整个索引会被序列化到磁盘并覆盖自身（无增量更新）
# 3. 数据库重启后需要将整个索引反序列化回内存，可能需要较长时间

HNSW 索引实验性持久化功能已启用: True
HNSW 索引实验性持久化功能已重置为: False


#### Inserts, Updates, Deletes and Re-Compaction

HNSW索引支持在创建索引后对表中的行进行插入、更新和删除。但是，需要注意以下两点：

- 在表中填充数据后创建索引会更快，因为初始批量加载可以更好地利用大型表上的并行性。
- 删除操作不会立即反映在索引中，而是被“标记”为已删除，这可能导致索引随时间变得陈旧，并对查询质量和性能产生负面影响。

为了解决最后一点，可以通过调用`PRAGMA hnsw_compact_index('index_name')`pragma函数来触发索引的重新压缩，修剪已删除的项，或者在大量更新后重新创建索引。

In [36]:
# 如果存在名为 my_vector_table_for_compact 的表，则删除它
conn.execute("""
DROP TABLE IF EXISTS my_vector_table_for_compact;
""")

# 创建一个名为 my_vector_table_for_compact 的新表
# 该表包含一个整数类型的 id 列和一个三维浮点数组类型的 vec 列
conn.execute("""
CREATE TABLE my_vector_table_for_compact (
    id INT,         -- 唯一标识符
    vec FLOAT[3]    -- 三维向量数据
);
""")

# 向 my_vector_table_for_compact 表中插入两条记录
conn.execute("""
INSERT INTO my_vector_table_for_compact 
VALUES 
    (1, [1.0, 2.0, 3.0]),  -- 插入第一条记录
    (2, [4.0, 5.0, 6.0]);  -- 插入第二条记录
""")

# 在 my_vector_table_for_compact 表的 vec 列上创建 HNSW 索引
conn.execute("""
CREATE INDEX my_hnsw_index_for_compact       -- 索引名称
ON my_vector_table_for_compact              -- 作用的表
USING HNSW (vec);                           -- 使用 HNSW 算法，作用于 vec 列
""")
print("已创建表 'my_vector_table_for_compact' 和索引 'my_hnsw_index_for_compact'。")

# 从 my_vector_table_for_compact 表中删除 id 为 1 的行
conn.execute("""
DELETE FROM my_vector_table_for_compact 
WHERE id = 1;  -- 删除条件
""")
print("已从 'my_vector_table_for_compact' 表中删除行。")

# 对名为 my_hnsw_index_for_compact 的 HNSW 索引执行重新压缩操作
# 这有助于移除被标记为已删除的条目，优化索引性能和准确性
conn.execute("""
PRAGMA hnsw_compact_index('my_hnsw_index_for_compact'); -- 指定要压缩的索引名称
""")
print("索引 'my_hnsw_index_for_compact' 已重新压缩。")

已创建表 'my_vector_table_for_compact' 和索引 'my_hnsw_index_for_compact'。
已从 'my_vector_table_for_compact' 表中删除行。
索引 'my_hnsw_index_for_compact' 已重新压缩。


#### Bonus: Vector Similarity Search Joins

`vss`扩展还提供了几个表宏，用于简化多个向量之间的匹配，即所谓的“模糊连接”。这些是：

- `vss_join(left_table, right_table, left_col, right_col, k, metric := 'l2sq')`
- `vss_match(right_table, left_col, right_col, k, metric := 'l2sq')`

这些**目前**还不使用`HNSW`索引，但作为方便用户使用的实用工具函数提供，对于那些可以接受不使用`HNSW`索引进行暴力向量相似性搜索，而不必自己编写连接逻辑的用户来说很有帮助。在未来，这些函数也可能会成为基于索引的优化目标。

这些函数的使用方法如下：

In [40]:
# 如果存在名为 haystack 的表，则删除它
conn.execute("""
-- 如果存在名为 haystack 的表，则删除它
DROP TABLE IF EXISTS haystack;
""")

# 如果存在名为 needle 的表，则删除它
conn.execute("""
-- 如果存在名为 needle 的表，则删除它
DROP TABLE IF EXISTS needle;
""")

# 创建名为 haystack 的表
# 该表包含一个整数类型的 id 列和一个三维浮点数组类型的 vec 列
conn.execute("""
-- 创建名为 haystack 的表
CREATE TABLE haystack (
    id INT,         -- 整数类型的ID列
    vec FLOAT[3]    -- 三维浮点数组类型的向量列
);
""")

# 创建名为 needle 的表
# 该表包含一个三维浮点数组类型的 search_vec 列，用于存储待搜索的向量
conn.execute("""
-- 创建名为 needle 的表
CREATE TABLE needle (
    search_vec FLOAT[3] -- 三维浮点数组类型的搜索向量列
);
""")

# 向 haystack 表中插入数据
# 使用 range 函数生成组合数据，并用 array_value 构建向量
# row_number() OVER () 用于生成唯一的ID
conn.execute("""
-- 向 haystack 表中插入数据
INSERT INTO haystack
SELECT
    row_number() OVER (),      -- 生成行号作为ID
    array_value(a, b, c)       -- 将 a, b, c 组合成一个三维浮点数组
FROM
    range(1, 10) ra(a),        -- 生成 1 到 9 的序列作为 a (不包含10)
    range(1, 10) rb(b),        -- 生成 1 到 9 的序列作为 b (不包含10)
    range(1, 10) rc(c);        -- 生成 1 到 9 的序列作为 c (不包含10)
                               -- 这将生成 9*9*9 = 729 条记录
""")

# 向 needle 表中插入两个搜索向量
conn.execute("""
-- 向 needle 表中插入数据
INSERT INTO needle
VALUES
    ([5, 5, 5]::FLOAT[3]),     -- 插入第一个搜索向量 [5.0, 5.0, 5.0]
    ([1, 1, 1]::FLOAT[3]);     -- 插入第二个搜索向量 [1.0, 1.0, 1.0]
""")
print("已创建并填充表 'haystack' 和 'needle'。")

# 使用 vss_join 宏执行向量相似性连接
# vss_join(左表, 右表, 左表向量列, 右表向量列, 返回近邻数k)
# res 是结果表的别名
join_df = conn.execute("""
-- 使用 vss_join 宏执行向量相似性连接
SELECT
    *
FROM
    vss_join(
        needle,       -- 左表 (包含搜索向量)
        haystack,     -- 右表 (包含待搜索的向量集合)
        search_vec,   -- 左表中的向量列名
        vec,          -- 右表中的向量列名
        3             -- 为左表中的每个向量返回右表中最近的 k=3 个向量
                      -- 默认使用 'l2sq' (欧氏距离平方) 度量
    ) res;            -- 结果集的别名
""").df()
print("\nvss_join 连接结果:")
print(join_df)

已创建并填充表 'haystack' 和 'needle'。

vss_join 连接结果:
   score                         left_tbl                            right_tbl
0    0.0  {'search_vec': (5.0, 5.0, 5.0)}  {'id': 365, 'vec': (5.0, 5.0, 5.0)}
1    1.0  {'search_vec': (5.0, 5.0, 5.0)}  {'id': 364, 'vec': (5.0, 4.0, 5.0)}
2    1.0  {'search_vec': (5.0, 5.0, 5.0)}  {'id': 356, 'vec': (5.0, 5.0, 4.0)}
3    0.0  {'search_vec': (1.0, 1.0, 1.0)}    {'id': 1, 'vec': (1.0, 1.0, 1.0)}
4    1.0  {'search_vec': (1.0, 1.0, 1.0)}   {'id': 10, 'vec': (1.0, 1.0, 2.0)}
5    1.0  {'search_vec': (1.0, 1.0, 1.0)}    {'id': 2, 'vec': (1.0, 2.0, 1.0)}


或者，可以使用`vss_match`宏作为“横向连接”，以按左表分组的方式获取匹配项。需要注意的是，这要求首先指定左表，然后是引用左表搜索列（在本例中为`search_vec`）的`vss_match`宏：

In [42]:
# 假设 'needle' 和 'haystack' 表已在先前单元格中创建并填充数据。
match_df = conn.execute("""
SELECT
    *  -- 选择所有列
FROM
    needle,  -- 从 'needle' 表 (包含我们要搜索的向量)
    vss_match(  -- 调用 vss_match 宏进行相似性搜索，它表现得像一个横向连接 (LATERAL JOIN)
        haystack,    -- 在 'haystack' 表中搜索
        search_vec,  -- 使用 'needle' 表中的 'search_vec' 列作为查询向量
        vec,         -- 'haystack' 表中被搜索的向量列是 'vec'
        3            -- 为每个 'search_vec' 返回最近的3个匹配项
    ) res;         -- 将 vss_match 的结果命名为 'res'
""").df()  # 将查询结果转换为 Pandas DataFrame

print("\nvss_match 函数执行结果:") # 打印 vss_match 的结果提示信息
print(match_df) # 打印包含匹配结果的 DataFrame


vss_match 函数执行结果:
        search_vec                                            matches
0  [5.0, 5.0, 5.0]  [{'score': 0.0, 'row': {'id': 365, 'vec': (5.0...
1  [1.0, 1.0, 1.0]  [{'score': 0.0, 'row': {'id': 1, 'vec': (1.0, ...


#### Limitations

- 目前仅支持由`FLOAT`（32位，单精度）组成的向量。
- 索引本身不进行缓冲管理，必须能够完全放入RAM内存中。
- 索引在内存中的大小不计入DuckDB的`memory_limit`配置参数。
- 除非将`SET hnsw_enable_experimental_persistence = ⟨bool⟩`配置选项设置为`true`，否则`HNSW`索引只能在内存数据库的表上创建，更多信息请参见“持久性”部分。
- 向量连接表宏（`vss_join`和`vss_match`）不需要也不使用`HNSW`索引。